In [1]:
%run Latex_macros.ipynb

<IPython.core.display.Latex object>

# Introduction


Up until now, the layers we have studed (Dense, Convolution) are primarily used to implement functions
that are one to one: 
- a single input (of fixed length) yields a single output.

Recurrent Neural Networks (RNN) deal with sequences: 
- sequences of inputs and sequence of outputs.

Sequences have order: a permutation of the elements of a sequence is a completely distinct sequence.

## Order matters !

Order matters
- set $\{ \x_{(1)} \dots \x_{(\tt-1)}\}$ versus sequence $\x_{(1)} \dots \x_{(\tt-1)}$
    - order doesn't matter for a set
    - $\{ \x_{(1)} \dots \x_{(\tt-1)}\} = \{  \x_{\tt-1)} \ldots \x_{(1)}\}$



<table>
    <tr>
        <th><center>Same prices</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_sequence_1.jpg" width=800></td>
    </tr>
</table>

<center>Same words</center>
$$
\begin{matrix}
\text{Machine} & \text{Learning} & \text{is} & \text{easy} & \text{not} & \text{difficult} \\
\text{Machine} & \text{Learning} & \text{is} & \text{difficult} & \text{not} & \text{easy} \\
\end{matrix}
$$


<table>
    <tr>
        <th><center>Same pixels</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_sequence_2.jpg" width=800></td>
    </tr>
</table>

All pairs of inputs in above examples are
- identical as sets
- different as sequences

**Note**

If the paired examples have different labels:
- NN will find as subset of features that separate them
- the separating feature then becomes an anchor, restricting reordering

## Functions on sequence

Examples of functions with sequence as inputs but single output (many to one mapping)
- predict next value in time series
- predict sentiment of text

Examples of functions with single input but sequence as output (one to many mapping)
- text generation (char-rnn) ?

Examples of functions with sequence as inputs and outputs
- translating from one language to another
- captioning a movie

**Notation alert**

Lot's of dimenions !
- $\x^\ip$ is a sequence with elements $\x^\ip_{(1)} \dots \x^\ip_{(\tt-1)}$
- each element $\x^\ip_\tp$ is a vector
    - $x^\ip_{(\tt),j}$ is a feature
        - examples
            - $\x^\ip_\tp$ is a frame $\tt$ in a movie; $x^\ip_{(\tt),j}$ is a pixel in frame $\tt$
            - $\x^\ip_\tp$ are the characteristics of a stock on day $\tt$, $x^\ip_{(\tt),j}$ is price or volume
            - $\x^\ip_\tp$ is a word $\tt$ in a sentence; $x^\ip_{(\tt),j}$ is element $j$ of the OHE of the word

Choices for how to predict $\y_\tp$ that is dependent on $\x_{(1)} \dots \x_\tp$

$$
\begin{array}[lll] \\
\pr{\y_\tp | \x_{(1)} \dots \x_\tp}  & \text{direct dependence on entire prefix } \x_{(1)} \dots \x_\tp \\
\\
\pr{\h_\tp | x_\tp, \h_{(\tt-1)} } & \text{latent variable } \h_\tp \text{encodes } \x_{(1)} \dots \x_\tp \\
\pr{\y_\tp | \h_\tp }              & \text{prediction contingent on latent variable} \\
\end{array}
$$

The Recurrent Neural Network (RNN) adopts the latter approach.

The single layer may also emit an output at step $i$ (for outputs that are sequences).

Here is some pseudo-code:

In [2]:
def RNN( input_sequence, state_size ):
    state = np.random.uniform(size=state_size)
    
    for input in input_sequence:
        # Consume one input, update the state
        out, state = f(input, state)
        
    return out
        

<table>
    <tr>
        <th><center>RNN</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_loop.jpg" width=1000></td>
    </tr>
</table>

Note that RNN's are sometimes drawn without separate outputs $\y_t$
- in that case, $\h_t$ may be considered the output. 

The computation of $\y_\tp$ will be just a linear transformation of $\h_t$ so there is no loss in omitting
it from the RNN and creating a separate node in the computation graph.

Geron does not distinguish betwee $\y_t$ and $\h_t$ and he uses the single $\y_\tp$ to denote the state.

I will use $\h$ rather than $\y$ to denote the "hidden state".


## $\h_\tp$ latent state

- $\h$ is a *fixed length* encoding of variable length sequence $\x_{(1)} \dots $
    - $\h_\tp$ encodes $\x_{(1)} \dots \x_\tp$
    - gives a way to have variable length input to, e.g., classifiers
- $\h_\tp$ is a vector of features
    - captures multiple "dimensions"/concepts of the input sequence


<table>
    <tr>
        <th><center>RNN Many to one; followed by classifier</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_many_to_one_to_classifier.jpg" width=800></td>
    </tr>
</table>

## Unrolling a loop
We can "unroll" the loop into the sequence of steps, with time as the horizontal axis

<table>
    <tr>
        <th><center>RNN unrolled</center></th>
    </tr>
    <tr>
        <td><img src="images/Unrolled_RNN.jpg" width=800></td>
    </tr>
</table>

Note that $\x, \y, \h$ are all vectors. 

In particular, the state $\h$ may have many elements
-  to record information about the entire prefix of the input.

The key connection is that the state at time $t-1$ is passed as input to time $t$.

So when processing $\x_\tp$
- the layer can take advantage of a summary ($\h_{(t-1)}$) of every input that preceded it.

One can look at this unrolled graph as being a dynamically-created computation graph.

Essentially, each state $\h$ is replicated per time step.

This view of the computation graph is important
- it shows you the exact computation
- it should tell you how gradients are computed
    - in particular, the loss and gradients flow backwards
        - so the gradients involving $\h$ are updated at *each* time step.
        - this will be important when we discuss
            - vanishing/exploding gradients
            - skip connections
            

# The RNN API

During one time step of computation, the RNN computes 2 values
- new  state $\h_\tp$
- output $\y_\tp$ (sometimes simply taken to be same as shor term state

The state computation is a function of the previous state $\h_{(t-1)}$,
and the current input $x_\tp$.

$$
\h_\tp = f(\x_\tp;  \h_{(t-1)})
$$

Note the recursive aspect of the computation of  $\h_\tp$: 
- it implicitly depends
on the values of the states at all previous time steps $t' < t$.


# RNN as a layer

## Many to one

Although the unrolled RNN looks confusing, as an "API" the RNN just acts as any other layer
- takes some input $\x$ (which happends to be a sequence)
- produces a single output

If we draw a box around the unrolled RNN, we can see the "API":

<table>
    <tr>
        <th><center>RNN many to one</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_many_one.jpg" width=800></td>
    </tr>
</table>


## RNN layer as an encoder
The many to one RNN essentially creates a compact encoding of an arbitrarily long sequence.

This can be very useful as we can feed this "summary" (representation) of the entire sequence
into layers that can't handle sequence inputs.

Note that there is nothing special about a layer creating a compact encoding (representation) of it's input.

A CNN layer, with outputs flattened to one dimension, creates a compact encoding of an image.

The real power of the RNN is the ability to encode all sequences, regardless of length, into a fixed size
representation.

### Sequences: variable length input summarized

$\h_\tp$ summarizes the length $\tt$ sequence $\x_{1,\ldots, \tt}$ in a *fixed size* vector $\h_\tp$.
- makes sequences amenable to models that can only deal with fixed size input


To be clear
- the RNN is a layer, just like any other
    - Internally it implements a loop but that is ordinarily hidden
    - The intuition about the "unrolled loop" is to help us to better understand the inner workings, not as a coding matter

- Like any other layer, it produces an output (although after multiple time steps for an RNN versus
a single time step for a Dense layer).
                                          
- If the length of sequence $\x$ is $T$, there is ordinarily a **single** output $\y_{(T)}$
    - $\y_{(T)}$ is only available after the entire input sequence has been consumed
    - the intermediate results 
    $$\h_\tp, \y_\tp, \; t = 1, \ldots, (T-1)$$ 
    are not visible through the API
    

## Many to many

The above behavior defines a many to one mapping from input sequence (many) to single output (one).

With a minor change, we can define a many to many mapping:
- each element of the input sequence
results in one element of an output sequence.

Many Deep Learning software API's will see recurrent layers with an optional
argument 
- `return_sequences`
- `return_states` 
- both default to `False` in Keras.

This controls the output behavior of the RNN layer, whether it returns one output per time step
$$
       \h_{(1)}, \ldots, \h_{(T)} \\
       \y_{(1)}, \ldots, \y_{(T)}
$$
or just
$$
\h_{(T)} \\
\y_{(T)}
$$

This is how any RNN behaves when the function it's implementing is many to many:
- one output per time step.

When the RNN needs to implement a many to one function, the layer looks like

<table>
    <tr>
        <th><center>RNN many to many</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_many_many.jpg" width=800></td>
    </tr>
</table>


The art-work needs to be clarified
- the RNN layer produces sequences
    - as outputs $\y$
    - as states $\h$

These sequences are available when the RNN layer *completes* its consumption of input $\x$.

The following diagram may clarify


<table>
    <tr>
        <th><center>RNN many to many, clarified</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_loop_many_many.jpg" width=1000></td>
    </tr>
</table>

- the `return_sequences` argument instructs the layer to produce a sequence $\y$
    - rather than a scalar, as in the many to one case
- the `return_states` argument instructs the layer to return the state $\h$ as well
    - useful if we stack RNN layers

# RNN details: update equations

$$
\begin{array}[lll]\\
\h_\tp & = & \phi(\W_{xh}\x_\tp  + \W_{hh}\h_{(t-1)}  + \b_h) \\
\y_\tp & = &  \W_{hy} \h_\tp  + \b_y \\
\end{array}
$$

where $\phi$ is an activation function (usually $\tanh$)

**Note** Geron prefers right multiplying weights $\x_\tp \W_{xh}$ versus $\W_{xh}\x_\tp$
- left multiplying seems more common in literature

**Note**
The equation is for a single example.  

In practice, we do an entire minibatch so have $m$ $\x's$
given as a $(m \times n)$ matrix $\mathbf{X}$.

**page 471: mention dimensions of each**

## Equation in pseudo-matrix form

You will often see a short-hand form of the equation.

Look at $\h_\tp$ as a function of two inputs $\x_, \h_{(t-1)}$.

We can stack the two inputs into a single matrix.

Stack the two matrices $\W_{xh}, \W_{hh}$ into a single weight matrix

$
\begin{array}[lll]\\
\h_\tp  = \W \mathbf{I} + \b \\
\text{ with } \\
\W = \left[
 \begin{matrix}
    \W_{xh} & \W_{hh}
 \end{matrix} 
 \right] \\
\mathbf{I} = \left[
 \begin{matrix}
    \x_\tp  \\
    \h_{(t-1)}
 \end{matrix} 
 \right] \\
\end{array}
$

# Back propagation through time (BPTT)

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>We can "unroll" the RNN into a sequence of layers, one per time step</li>
        <li>In theory: Back Propagation on the unrolled RNN is the same as for a non-Recurrent Network</li>
        <li>In practice: the unrolled RNN is very deep, which causes issues in Back Propagation.
    </ul>
</div>


*Back Propagation Through Time (BPTT)* refers to
- unrolling the RNN computation into a sequence of layers
- performing ordinary Back Propagation in order to update weights

- In a non-Recurrent network:
    - $\W_\llp$, the weights of layer $\ll$, affect only layers $\ll$ and greater.
    - This means the backward flow of the gradient with respect to $\W_\llp$ stops at layer $\ll$.
- In  Recurrent Network:
    - All unrolled "layers" share the *same* weights
    - This means the gradients with respect to shared weight $\W$ must flow backward all the way to the input layer at time $0$.
    

<table>
    <tr>
        <th><center>RNN Loss Gradient Flow</center></th>
    </tr>
    <tr>
        <td><img src="images/RNN_layer_loss_gradient.jpg" width=800></td>
    </tr>
</table>

The unrolled graph is as deep as the length of $\x^\ip$ ($T^\ip = |\x^\ip|)$ 
- weights can update only after $T^\ip$ input values have been processed,
so training can be slow.
- Vanishing Gradients become a concern for large $T^\ip$
    - Recall from the Vanishing Gradient lecture: magnitude of gradients diminishes from layer $\ll$ to layer $(\ll-1)$ during back propagation

# RNN vanishing/exploding gradient problem


<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>A "single-layer RNN that has been unrolled for T time steps</li>
        <ul>
            <li>is mathematically equivalent to a simple NN with T layers</li>
            <li><bold>BUT</bold> all layers share the same weights</li>
        </ul>
        <li>This sharing of weights leads to a problem of Vanishing/Exploding gradients</li>
        <ul>
            <li>Similar to the vanishing gradient problem we derived for simple NN</li>
            <li>but with a different root cause (weight sharing)</li>
        </ul>
    </ul>
</div>

<div class="alert alert-block alert-warning">
    <b>TL;DR</b> 
    <br>
    <ul>
        <li>Why shared weights are different</li>
        <ul>
            <li>Output <bold>y</bold> at time t is a function of cell state h at time t</li>
            <li>Cell state h at time t is recursively defined</li>
            <ul>
                <li>So it is a function of cell states over all times t' < t as well</li>
                <li>This means the weight update involves a repeated product: (t -t') times</li>
                <li>This product tends to 0 (vanishing) or infinity (explode) as (t -t') increases</li>
            </ul>
            <li>So losses at time step t have difficulty updating gradients for the distant past<</li>
            <li>RNN has difficulty with long-term dependencies</li>   
        </ul>
    </ul>
</div>

Returning to the loss gradient
we encountered the terms

$$\frac{\partial \y^\ip_\tp}{\partial \W}$$

We will focus on the part of $\W$ that is $\W_{hh}$

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp} \frac{{\partial \h_\tp}}{\partial \W_{hh}}
$$

But recursively defined $\h_\tp$ is a function of $\h_{(\tt-1)}, \h_{(\tt-1)}, \ldots, \h_{(1)}$
so

$$
\frac{\partial \y_\tp}{\partial W_{hh}} = \frac{\partial \y_\tp}{\partial \h_\tp}
 \sum_{k=0}^\tt {  \frac{\partial \h_\tp}{\partial \h_{(\tt -k)}} \frac{\partial \h_{(\tt -k)}}{\partial \W_{hh}} }
$$

The summation: $\frac{\partial \h_\tp}{\partial \W_{hh}}$, through all intermediate $\h_{(\tt -k)}$


The problematic term for us is
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}}
$$

It can be computed by the Chain Rule as
$$
\frac{\partial \h_\tp}{\partial \h_{(\tt-k)}} = \prod_{u=0}^{\tt-1} { \frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u -1) }}}
$$

Each term
$$
\frac{\partial \h_{(\tt -u)}}{\partial \h_{(\tt - u-1)}}
$$
results in a term $\W_{hh}$ so the repeated product compute matrix $\W_{hh}$ raised to the power $k$.



For simplicity,  suppose $\W_{hh}$ were a scalar
- if $\W_{hh} < 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $0$
- if $\W_{hh} > 1$ then repeatedly multiply $\W_{hh}$ by itself approaches $\infty$

In other words:
- as the distance between time steps $\tt$ and $(\tt -k)$ increases
- the gradient (for the weight  update) either vanishes or explodes.

Since this term is used in the update for our weights
- updates  will either be erratic (too big) 
- or
non-existent, hampering learning of weights.

This was not necessarily a problem in non-recurrent networks 
- because each layer had a different
weight matrix.

What an RNN does that helps it be parsimonius in number of parameters
- by sharing the weights across
all time steps
- hurts us in learning.
                                              

For the general case where $\W_{hh}$ is a matrix
- we can show the same resul with the eigenvalues of the matrix

# Visualization of RNN hidden state

Here is a [visualization](http://karpathy.github.io/2015/05/21/rnn-effectiveness/#visualizing-the-predictions-and-the-neuron-firings-in-the-rnn) of single elements within the hidden state, as they consume the input sequence
of *single characters*.

The color reflects the intensity (value) of the paricular cell (blue=low, red=high)


<table>
    <tr>
        <th><center>State activations after seeing prefix of input</center></th>
    </tr>
    <tr>
        <td><img src="images/Unreasonable_effectiveness_1.png" width=800></td>
    </tr>
</table>

In [3]:
print("Done")

Done
